In [111]:
import warnings
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('D:/工作/data/shibie/')
warnings.filterwarnings('ignore')
train_data = pd.read_csv('train_data.csv')
test =pd.read_csv('test.csv')
train_data = train_data.set_index('ID')
test = test.set_index('ID')
Idx = train_data.index
testIdx = test.index
data = pd.concat([train_data,test])

In [112]:
#统计每个样本的缺失情况。
train_data['null'] = train_data.isnull().sum(axis=1)
train_data['null'] = (train_data['null']>50).astype(int)
test['null'] = test.isnull().sum(axis=1)
test['null'] = (test['null']>50).astype(int)

In [9]:
mask_null = train_data['null'] == 1
train_data.loc[mask_null,'Label'].value_counts()

0    7434
1     909
Name: Label, dtype: int64

In [10]:
train_data.loc[~mask_null,'Label'].value_counts()

0    5991
1      32
Name: Label, dtype: int64

通过上面的分析，对于确实较少的样本，其负样本率非常低， 所以这个可以作为一条规则，确实特征小于某个值可以直接判定为好，我们将对那些确实严重的样本进行建模。

In [114]:
# 删除掉缺失率为1的特征。
columns = [each for each in data.columns if '年' in each]
data1 = data.drop(columns,axis=1)

In [31]:
z = pd.qcut(train_data[columns[50]],10,duplicates='drop',retbins=True)

In [123]:
for each in columns:
    if train_data[each].nunique()==1:
        continue
    ser,bins = pd.qcut(train_data[each], 10, duplicates='drop', retbins=True)
    bins[0],bins[-1] = -np.inf, np.inf
    ser.cat.categories = range(len(ser.cat.categories))
    train_data[each+'_bin'] = ser.astype(float).fillna(-1)
    ser2 = pd.cut(test[each],bins,labels=range(len(bins)-1))
    test[each+'_bin'] = ser2.astype(float).fillna(-1)

In [124]:
bincolumns = [each for each in train_data.columns if 'bin' in each]

138

In [115]:
data1 = data1.drop('投资总额',axis=1)

In [116]:
def Tax(data):
    train_data = data.copy()
    tax=['增值税','企业所得税','印花税','教育费','城建税', '注册资本']
    traintax = train_data[tax]
    traintax.fillna(0)
    traintax['zero']=(traintax==0).sum(axis=1)
    traintax['taxsum'] = traintax.iloc[:,:5].sum(axis=1)
    traintax['taxmean'] = traintax.mean(axis=1)
    traintax['median'] = traintax.median(axis=1)
    traintax = traintax.drop(tax,axis=1)
    return traintax
traintax = Tax(data1)

In [117]:
time = ['经营期限至','注销时间','经营期限自','成立日期','核准日期']
for each in time:
    data1[each+'_0'] = data1[each].str.split(
        ':').str[0].astype(float)
    data1[each+'_1'] = data1[each].str.split(
        ':').str[0].astype(float)
data1 = data1.drop(time,axis=1)

data1['经营范围']=data1['经营范围'].str.split(',').apply(len)

data1['邮编1']=data1['邮政编码'].astype(str).str[:2]
data1['邮编2']=data1['邮政编码'].astype(str).str[2:4] 
mask = (data1['邮编1']=='na')|(data1['邮编1']=='26')
data1.loc[~mask,'邮编1'] = '10'
youbian = data1['邮编2'].value_counts()[:9].index
data1.loc[~data1['邮编2'].isin(youbian),'邮编2'] = '10'
you2 = pd.get_dummies(data1['邮编2'],prefix='邮编2')
you1 = pd.get_dummies(data1['邮编1'],prefix='邮编1')
data1 = data1.drop(['邮政编码','邮编1','邮编2'],axis=1)
#相关系数证明与类别无关。
# data1 = data1.drop(['登记机关','管辖机关'],axis=1)
#行业代 
temp = data1['行业代码'].fillna(0).astype(int).astype(str).str.ljust(4,'0')
temp1 = temp.str[:2]
temp2 = temp.str[2]
temp3 = temp.str[3]
daima1 = pd.get_dummies(temp1,prefix='代码1')
daima2 = pd.get_dummies(temp2,prefix='代码2')
daima3 = pd.get_dummies(temp3,prefix='代码3')

# data2 = data1.drop('行业代码',axis=1)
data2 = pd.concat([data1,daima1,daima2,daima3,you1,you2],axis=1)

In [118]:
data2 = pd.concat([data2,traintax],axis=1)

In [125]:
z = train_data[bincolumns].corrwith(train_data['Label'])

In [121]:
data2.shape

(23944, 145)

In [126]:
X = data2.loc[Idx,:]
X['null'] = train_data.isnull().sum(axis=1)
Xv = data2.loc[testIdx,:]
Xv['null'] = test.isnull().sum(axis=1)

In [127]:
X_ = pd.concat([X,train_data[bincolumns]],axis=1)
Xv_ = pd.concat([Xv,test[bincolumns]],axis=1)

In [128]:
Xv = Xv.drop('Label',axis=1)

In [129]:
X_.to_csv('X.csv',index=False)
Xv_.to_csv('Xv.csv',index=False)

In [105]:
sub = pd.read_csv('sub.csv')

In [106]:
sub.columns=['ID','Label']

In [108]:
sub.to_csv('sub2.csv',index=False)

In [107]:
sub['ID'] = test.index

In [109]:
sub.head()

,ID,Label
0,0,0.071114
1,2,0.133089
2,4,0.092748
3,5,0.077130
4,6,0.077572


In [113]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
model = lgb.LGBMClassifier(**params)
cross_val_score(model,X.to_numpy(),y,
                scoring='roc_auc',cv=StratifiedKFold(5,shuffle=True))

array([0.91642773, 0.90728337, 0.91169817, 0.9128561 , 0.91469947])

In [109]:
model.fit(X.to_numpy(),y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=50, n_jobs=-1, num_leaves=10, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=5)

In [110]:
model.feature_importances_

array([65,  1,  3, 10, 34, 17, 25, 29,  9, 16,  9,  7, 12,  0,  1,  8,  2,
        9,  2,  4, 12,  0,  0,  0,  0,  3,  0,  2,  0,  0,  0,  0,  0,  0,
        4,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  1,  1,
        0,  0,  0,  0,  2,  4,  0,  2,  4,  0,  0,  1,  1,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  3,  0,  1,  0,  0,  0,  0,  0,  0,  1,  8,  0,  0,  0,
        0,  3,  0,  0,  1,  0,  1,  1,  0,  3,  0,  2,  2,  0,  0,  2,  0,
        0, 12, 28, 24, 17, 34])

In [ ]:
X2 = X.loc[~mask_null,:]
y2 = X2.pop('Label')

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
model2 = lgb.LGBMClassifier(**params)
cross_val_score(model2,X2.to_numpy(),y2,
                scoring='roc_auc',cv=StratifiedKFold(5,shuffle=True,
                                        random_state=10))

In [59]:
model2.fit(X2.to_numpy(),y2)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=50, n_jobs=-1, num_leaves=10, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=5)

In [111]:
params = {'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.5,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 10,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1,
 'subsample_for_bin': 200000,
 'subsample_freq': 5}

In [60]:
Test = data2.loc[testIdx,:]
Test = Test.drop('Label',axis=1)

In [54]:
test_mask_null = test['null'] == 1

In [66]:
Test1 = Test.loc[test_mask_null,:]

In [67]:
Test2 = Test.loc[~test_mask_null,:]

In [79]:
#Test1['Label']= model.predict_proba(Test1.to_numpy())[:,1]
Test2['Label'] = 0

In [80]:
sub = pd.concat([Test1,Test2])
sub['Label'].reset_index().to_csv('sub.csv',index=False)

In [76]:
Test1['Label'].min(),Test1['Label'].max()

(0.002711315428739353, 0.5633296665704199)

In [75]:
Test2['Label'].min(),Test2['Label'].max()

(7.930796402812578e-05, 0.6637875725646716)

In [77]:
np.percentile(Test2['Label'],90)

0.0046281099724601125

In [ ]:
shrunk_cov.flat[::n_features + 1] += shrinkage * mu